## Importing libraries

In [1]:
import numpy as np
import pandas as pd
import cv2
from tensorflow.keras.models import load_model
import os
import imutils
from imutils.video import VideoStream

## Loading model and labels

In [2]:
LABELS = "labels.csv"
model = load_model("traffic_sign_classfier.model")
labelData=pd.read_csv(LABELS)

## Utility function

In [3]:
def grayscale(img):
  return cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)

def equilize(img):
  return cv2.equalizeHist(img)

def preprocessing(img):
  img = grayscale(img)
  img = equilize(img)
  img = img/255.
  return img

def getClassName(classNo):
    return labelData.Name[classNo]

## Realtime Traffic sign recognition

In [32]:
def cam():
    vs = VideoStream(src=0).start()
    
    while True:
        frame = vs.read()
        frame = imutils.resize(frame, width=1080)
        img = np.asarray(frame)
        img = cv2.resize(img,(32,32))
        img = preprocessing(img)
        cv2.imshow("Preprocessed Image",img)
        img = img.reshape(1,32,32,1)
        font = cv2.FONT_HERSHEY_SIMPLEX
        cv2.putText(frame, "CLASS: " , (20, 35), font, 0.75, (0, 0, 255), 2, cv2.LINE_AA)
        cv2.putText(frame, "PROBABILITY: ", (20, 75), font, 0.75, (0, 0, 255), 2, cv2.LINE_AA)
        
        predictions = model.predict(img)
        probabilityValue =np.amax(predictions)
        classIndex = np.argmax(predictions)
        
        threshold = 0.75
        if probabilityValue > threshold:
            cv2.putText(frame,str(classIndex)+"-"+str(getClassName(classIndex)),(120, 35), font, 0.75, (0, 0, 255), 2, cv2.LINE_AA)
            cv2.putText(frame, str(round(probabilityValue*100,2) )+"%", (180, 75), font, 0.75, (0, 0, 255), 2, cv2.LINE_AA)
        cv2.imshow("Result",frame)
        
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break
    cv2.destroyAllWindows()
    vs.stop()

In [33]:
cam()

In [36]:
def filteringImages(img):
    img=cv2.GaussianBlur(img,(11,11),0)
    return img

def returnRedness(img):
	yuv=cv2.cvtColor(img,cv2.COLOR_BGR2YUV)
	y,u,v=cv2.split(yuv)
	return v

def threshold(img,T=150):
	_,img=cv2.threshold(img,T,255,cv2.THRESH_BINARY)
	return img 

def findContour(img):
	contours, hierarchy = cv2.findContours(img,cv2.RETR_TREE,cv2.CHAIN_APPROX_SIMPLE)
	return contours

def findBiggestContour(contours):
	m=0
	c=[cv2.contourArea(i) for i in contours]
	return contours[c.index(max(c))]

def boundaryBox(img,contours):
	x,y,w,h=cv2.boundingRect(contours)
	img=cv2.rectangle(img,(x,y),(x+w,y+h),(0,255,0),2)
	sign=img[y:(y+h) , x:(x+w)]
	return img,sign

In [55]:
def cam_test():
    vs = VideoStream(src=0).start()
    while True:
        frame = vs.read()
        frame = imutils.resize(frame,width=1080)
        thresh = threshold(frame,T=150)
        try:
            contours = findContour(thresh)
            big = findBiggestContour(contours)
#             if cv2.contourArea(big)>1:
            img,sign = boundaryBox(frame,big)
            cv2.imshow('frame',sign)
#             else:
#                 cv2.imshow('frame',frame)
        except:
            cv2.imshow('frame',frame)
        
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break
    cv2.destroyAllWindows()
    vs.stop()

In [56]:
cam_test()